# Z Boson Analysis using a Boosted Decision Tree, with TMVA and JupyROOT

This notebook uses the available Atlas Open Data http://opendata.atlas.cern, to study the Z boson production in the two-lepton final state http://opendata.atlas.cern/release/2020/documentation/physics/DL1.html. 

Here, a full analysis of the Z boson production is made, filtering and selecting the events in order to satisfy the selection criteria. The goal is to process the results with the TMVA, in order to train a Boosted Decision Tree with a fraction of the samples. This tool returns back a BDT variable which allows to cut on events tagged as background, that is, minimizing the amount of background, so we just keep signal information.

TMVA requires separate samples for signal, background and data, then a first analysis is made just for the simulated signal samples, filtering the events to keep the best information of the process, and saving it in a root file. The analysis for background and data is similar in both cases, just recreating the interest variables, and discargind some useless events, but not filtering as much as in the signal section.

This notebook has the advantage of being simple to understand and allows to have an easy control over the samples that are being used, also it do not requires variable initialization. 


In [2]:
import ROOT
from ROOT import TMath, TChain
import time
import uproot3
import numpy as np
import root_pandas as rp
import infofile

## Signal Section

First, we select and extract the signal samples from the simulated MC samples

In [2]:
tree = TChain("mini") #Create a chain to store the samples
tree.Add("https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/2lep/MC/mc_361106.Zee.2lep.root")
t1=tree.GetEntries()
tree.Add("https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/2lep/MC/mc_361107.Zmumu.2lep.root")
t2=tree.GetEntries()
tree.Add("https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/2lep/MC/mc_361108.Ztautau.2lep.root")
t3=tree.GetEntries()
samplesSig = [t1,t2,t3]
print(samplesSig)

In [4]:
#create the target root file 
roofs = uproot3.recreate("RootFiles/signal.root")
roofs

<TFileRecreate b'signal.root' at 0x7f2089f83670>

Inside target root file, we open the interest variables, in this case we just keep the invariant mass from the two lepton system, and the Pt of the leading and subleading leptons

In [5]:
roofs["signal"] = uproot3.newtree({"mll": np.float32, "lead": np.float32, "sublead": np.float32, "weight": np.float32})

In [6]:
start = time.time()
#list to store information of good events
invM=[]
leadpt=[]
subleadpt=[]
weights=[]
leadlepton = ROOT.TLorentzVector()
subleadlepton = ROOT.TLorentzVector()
lepton12 = ROOT.TLorentzVector()
k=0
kf=0
lumi=10
index1=0
index2=0
print("Running a total of ",samplesSig[2]," events")
for event in tree:
    k=k+1
    goodlep=0
    #if you want to reduce computation times, you can stop the process after a number of events
    #if(k==10000000): break #just make sure you obtain enough final events.
    
    #start cutting on events
    if(tree.trigE or tree.trigM):
        if(tree.lep_n==2):
            for j in range(tree.lep_n):
                if((tree.lep_pt[j]>25000.) and (tree.lep_ptcone30[j]/tree.lep_pt[j] < 0.15) and (tree.lep_etcone20[j]/tree.lep_pt[j] < 0.15)):
                    #Good electron
                    if(tree.lep_type[j]==11 and abs(tree.lep_eta[j])<2.47 and (abs(tree.lep_eta[j])<1.37 or abs(tree.lep_eta[j])>1.52)):
                        theta = 2*np.arctan(np.exp(-tree.lep_eta[j]))
                        if(tree.lep_trackd0pvunbiased[j]/tree.lep_tracksigd0pvunbiased[j] < 5 and abs(tree.lep_z0[j]*np.sin(theta))<0.5):
                            goodlep+=1
                            index1=j
                    #Good muon
                    if(tree.lep_type[j]==13 and abs(tree.lep_eta[j]<2.5)):
                        theta = 2*np.arctan(np.exp(-tree.lep_eta[j]))
                        if(tree.lep_trackd0pvunbiased[j]/tree.lep_tracksigd0pvunbiased[j] < 3 and abs(tree.lep_z0[j]*np.sin(theta))<0.5):
                            goodlep+=1
                            index2=j
            if(goodlep==2):
                if(tree.jet_n==0):
                    if(tree.lep_charge[index1]*tree.lep_charge[index2] < 0 and tree.lep_type[index1]==tree.lep_type[index2]):
                        if(tree.lep_pt[index1]>tree.lep_pt[index2]):
                            leadlepton.SetPtEtaPhiE(tree.lep_pt[index1], tree.lep_eta[index1], tree.lep_phi[index1], tree.lep_E[index1])
                            subleadlepton.SetPtEtaPhiE(tree.lep_pt[index2], tree.lep_eta[index2], tree.lep_phi[index2], tree.lep_E[index2])
                        else:
                            leadlepton.SetPtEtaPhiE(tree.lep_pt[index2], tree.lep_eta[index2], tree.lep_phi[index2], tree.lep_E[index2])
                            subleadlepton.SetPtEtaPhiE(tree.lep_pt[index1], tree.lep_eta[index1], tree.lep_phi[index1], tree.lep_E[index1])
                        lepton12=leadlepton+subleadlepton
                        invmass=lepton12.M()/1000.
                        if(abs(invmass-91.18)<25.):
                            ptlead=leadlepton.Pt()/1000.
                            ptsub=subleadlepton.Pt()/1000.
                            invM.append(invmass)
                            leadpt.append(ptlead)
                            subleadpt.append(ptsub)
                            #Here, it is necessary to check from which process does the event come
                            #Because we are using a TChain, the only way to determine the origin of the event is by its lenght
                            #This is done with the sizes of each sample. These were store in SamplesSig
                            if(k<=samplesSig[0]):
                                sample='Zee'
                            if(k>samplesSig[0] and k<=samplesSig[1]):
                                sample='Zmumu'
                            if(k>samplesSig[1] and k<=samplesSig[2]):
                                sample='Ztautau'
                            info = infofile.infos[sample]
                            xsec_weight = (lumi*1000*info["xsec"])/(info["sumw"]*info["red_eff"])
                            weight=xsec_weight*tree.mcWeight*tree.scaleFactor_PILEUP*tree.scaleFactor_ELE*tree.scaleFactor_MUON*tree.scaleFactor_LepTRIGGER
                            weights.append(weight)
                            kf+=1
    #Time control section
    if(k==10000000):
        xd=time.time()
        taked=xd-start
        tre=(samplesSig[2]-k)*taked/k
        print("10M completed in {} min {} s".format(int(taked//60),int(taked%60)))
        print("Approx time for finishing is {} min {} s".format(int(tre//60),int(tre%60))) 
    if(k==20000000):
        xd=time.time()
        taked=xd-start
        tre=(samplesSig[2]-k)*taked/k
        print("20M completed in {} min {} s".format(int(taked//60),int(taked%60)))
        print("Approx time for finishing is {} min {} s".format(int(tre//60),int(tre%60))) 
    if(k==30000000):
        xd=time.time()
        taked=xd-start
        tre=(samplesSig[2]-k)*taked/k
        print("30M completed in {} min {} s".format(int(taked//60),int(taked%60)))
        print("Approx time for finishing is {} min {} s".format(int(tre//60),int(tre%60))) 
    if(k==40000000):
        xd=time.time()
        taked=xd-start
        tre=(samplesSig[2]-k)*taked/k
        print("40M completed in {} min {} s".format(int(taked//60),int(taked%60)))
        print("Approx time for finishing is {} min {} s".format(int(tre//60),int(tre%60))) 

end = time.time()
duration = end-start
print('Initial events:', k)
print('Final events:', kf)
print("Finished in {} min {} s".format(int(duration//60),int(duration%60)))
#writing and closing root file
roofs["signal"].extend({"mll": invM, "lead": leadpt, "sublead": subleadpt, "weight": weights})
roofs.close()
print('Successful close')

Running a total of  44153520  events
10M completed in 34 min 18 s
Approx time for finishing is 117 min 11 s
20M completed in 69 min 49 s
Approx time for finishing is 84 min 19 s
30M completed in 106 min 13 s
Approx time for finishing is 50 min 6 s
40M completed in 142 min 46 s
Approx time for finishing is 14 min 49 s
Initial events: 44153520
Final events: 8498254
Finished in 157 min 38 s
Successful close


## Now the background section

We also extract the background samples from the MC samples

In [11]:
treeb = TChain("mini")
treeb.Add("https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/2lep/MC/mc_363356.ZqqZll.2lep.root")
b1=treeb.GetEntries()
treeb.Add("https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/2lep/MC/mc_363358.WqqZll.2lep.root")
b2=treeb.GetEntries()
treeb.Add("https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/2lep/MC/mc_363359.WpqqWmlv.2lep.root")
b3=treeb.GetEntries()
treeb.Add("https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/2lep/MC/mc_363360.WplvWmqq.2lep.root")
b4=treeb.GetEntries()
treeb.Add("https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/2lep/MC/mc_363489.WlvZqq.2lep.root")
b5=treeb.GetEntries()
treeb.Add("https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/2lep/MC/mc_363490.llll.2lep.root")
b6=treeb.GetEntries()
treeb.Add("https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/2lep/MC/mc_363491.lllv.2lep.root")
b7=treeb.GetEntries()
treeb.Add("https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/2lep/MC/mc_363492.llvv.2lep.root")
b8=treeb.GetEntries()
treeb.Add("https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/2lep/MC/mc_363493.lvvv.2lep.root")
b9=treeb.GetEntries()
treeb.Add("https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/2lep/MC/mc_410011.single_top_tchan.2lep.root")
b10=treeb.GetEntries()
treeb.Add("https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/2lep/MC/mc_410012.single_antitop_tchan.2lep.root")
b11=treeb.GetEntries()
treeb.Add("https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/2lep/MC/mc_361100.Wplusenu.2lep.root")
b12=treeb.GetEntries()
samplesBck=[b1,b2,b3,b4,b5,b6,b7,b8,b9,b10,b11,b12]

Again, open a root file to store the events from the background processes

In [12]:
roofb = uproot3.recreate("RootFiles/background.root")
roofb

<TFileRecreate b'background.root' at 0x7f92ce5b0e50>

In [13]:
roofb["background"] = uproot3.newtree({"mll": np.float32, "lead": np.float32, "sublead": np.float32, "weight": np.float32})

In [14]:
#This analysis section is similar to the signal section, but is less strict when cleaning events.
start = time.time()
invMb=[]
leadptb=[]
subleadptb=[]
weightsb=[]
leadlepton = ROOT.TLorentzVector()
subleadlepton = ROOT.TLorentzVector()
lepton12 = ROOT.TLorentzVector()
k=0
kf=0
lumi=10
index1=0
index2=1
print("Running a total of ",samplesBck[11]," events")
for event in treeb:
    k=k+1
    if(treeb.trigE or treeb.trigM):
        if(treeb.lep_n==2):
            if(treeb.lep_pt[index1]>treeb.lep_pt[index2]):
                leadlepton.SetPtEtaPhiE(treeb.lep_pt[index1], treeb.lep_eta[index1], treeb.lep_phi[index1], treeb.lep_E[index1])
                subleadlepton.SetPtEtaPhiE(treeb.lep_pt[index2], treeb.lep_eta[index2], treeb.lep_phi[index2], treeb.lep_E[index2])
            else:
                leadlepton.SetPtEtaPhiE(treeb.lep_pt[index2], treeb.lep_eta[index2], treeb.lep_phi[index2], treeb.lep_E[index2])
                subleadlepton.SetPtEtaPhiE(treeb.lep_pt[index1], treeb.lep_eta[index1], treeb.lep_phi[index1], treeb.lep_E[index1])
            lepton12=leadlepton+subleadlepton
            invmass=lepton12.M()/1000.
            ptlead=leadlepton.Pt()/1000.
            ptsub=subleadlepton.Pt()/1000.
            invMb.append(invmass)
            leadptb.append(ptlead)
            subleadptb.append(ptsub)
            if(k<=samplesBck[0]):
                sample='ZqqZll'
            if(k>samplesBck[0] and k<=samplesBck[1]):
                sample='WqqZll'
            if(k>samplesBck[1] and k<=samplesBck[2]):
                sample='WpqqWmlv'
            if(k>samplesBck[2] and k<=samplesBck[3]):
                sample='WplvWmqq'
            if(k>samplesBck[3] and k<=samplesBck[4]):
                sample='WlvZqq'
            if(k>samplesBck[4] and k<=samplesBck[5]):
                sample='llll'
            if(k>samplesBck[5] and k<=samplesBck[6]):
                sample='lllv'
            if(k>samplesBck[6] and k<=samplesBck[7]):
                sample='llvv'
            if(k>samplesBck[7] and k<=samplesBck[8]):
                sample='lvvv'
            if(k>samplesBck[8] and k<=samplesBck[9]):
                sample='single_top_tchan'
            if(k>samplesBck[9] and k<=samplesBck[10]):
                sample='single_antitop_tchan'
            if(k>samplesBck[10] and k<=samplesBck[11]):
                sample='Wplusenu'
            info = infofile.infos[sample]
            xsec_weight = (lumi*1000*info["xsec"])/(info["sumw"]*info["red_eff"])
            weight=xsec_weight*treeb.mcWeight*treeb.scaleFactor_PILEUP*treeb.scaleFactor_ELE*treeb.scaleFactor_MUON*treeb.scaleFactor_LepTRIGGER
            weightsb.append(weight)
            kf+=1
                            

    if(k==5000000):
        xd=time.time()
        taked=xd-start
        tre=(samplesBck[11]-k)*taked/k
        print("5M completed in {} min {} s".format(int(taked//60),int(taked%60)))
        print("Approx time for finishing is {} min {} s".format(int(tre//60),int(tre%60))) 
    if(k==10000000):
        xd=time.time()
        taked=xd-start
        tre=(samplesBck[11]-k)*taked/k
        print("10M completed in {} min {} s".format(int(taked//60),int(taked%60)))
        print("Approx time for finishing is {} min {} s".format(int(tre//60),int(tre%60))) 
    if(k==14000000):
        xd=time.time()
        taked=xd-start
        tre=(samplesBck[11]-k)*taked/k
        print("14M completed in {} min {} s".format(int(taked//60),int(taked%60)))
        print("Approx time for finishing is {} min {} s".format(int(tre//60),int(tre%60))) 

end = time.time()
duration = end-start
print('Initial events:', k)
print('Final events:', kf)
print("Finished in {} min {} s".format(int(duration//60),int(duration%60)))
roofb["background"].extend({"mll": invMb, "lead": leadptb, "sublead": subleadptb, "weight": weightsb})
roofb.close()
print('Successful close')

Running a total of  14371616  events
5M completed in 8 min 13 s
Approx time for finishing is 15 min 24 s
10M completed in 15 min 24 s
Approx time for finishing is 6 min 44 s
14M completed in 22 min 30 s
Approx time for finishing is 0 min 35 s
Initial events: 14371616
Final events: 11041114
Finished in 23 min 18 s
Successful close


## Now, we extract and merge the data into one root file

Here, we do the exact same process but with the data.

In [4]:
data = TChain("mini")
data.Add("https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/2lep/Data/data_A.2lep.root")
data.Add("https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/2lep/Data/data_B.2lep.root")
data.Add("https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/2lep/Data/data_C.2lep.root")
data.Add("https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/2lep/Data/data_D.2lep.root")
d=data.GetEntries()
print(d)

12205790


In [5]:
roofd = uproot3.recreate("RootFiles/data.root")
roofd

<TFileRecreate b'data.root' at 0x7f92cef6c040>

In [6]:
roofd["data"] = uproot3.newtree({"mll": np.float32, "lead": np.float32, "sublead": np.float32, "weight": np.float32})

In [8]:
start = time.time()
invMd=[]
leadptd=[]
subleadptd=[]
weightsd=[]
leadlepton = ROOT.TLorentzVector()
subleadlepton = ROOT.TLorentzVector()
lepton12 = ROOT.TLorentzVector()
k=0
kf=0
lumi=10
index1=0
index2=1
print("Running a total of ",d," events")
for event in data:
    k=k+1
    if(data.trigE or data.trigM):
        if(data.lep_n==2):
            if(data.lep_pt[index1]>data.lep_pt[index2]):
                leadlepton.SetPtEtaPhiE(data.lep_pt[index1], data.lep_eta[index1], data.lep_phi[index1], data.lep_E[index1])
                subleadlepton.SetPtEtaPhiE(data.lep_pt[index2], data.lep_eta[index2], data.lep_phi[index2], data.lep_E[index2])
            else:
                leadlepton.SetPtEtaPhiE(data.lep_pt[index2], data.lep_eta[index2], data.lep_phi[index2], data.lep_E[index2])
                subleadlepton.SetPtEtaPhiE(data.lep_pt[index1], data.lep_eta[index1], data.lep_phi[index1], data.lep_E[index1])
            lepton12=leadlepton+subleadlepton
            invmass=lepton12.M()/1000.
            ptlead=leadlepton.Pt()/1000.
            ptsub=subleadlepton.Pt()/1000.
            invMd.append(invmass)
            leadptd.append(ptlead)
            subleadptd.append(ptsub)
            weightsd.append(1)
            kf+=1
    if(k==5000000):
        xd=time.time()
        taked=xd-start
        tre=(d-k)*taked/k
        print("5M completed in {} min {} s".format(int(taked//60),int(taked%60)))
        print("Approx time for finishing is {} min {} s".format(int(tre//60),int(tre%60))) 
    if(k==10000000):
        xd=time.time()
        taked=xd-start
        tre=(d-k)*taked/k
        print("10M completed in {} min {} s".format(int(taked//60),int(taked%60)))
        print("Approx time for finishing is {} min {} s".format(int(tre//60),int(tre%60))) 
end = time.time()
duration = end-start
print('Initial events:', k)
print('Final events:', kf)
print("Finished in {} min {} s".format(int(duration//60),int(duration%60)))
roofd["data"].extend({"mll": invMd, "lead": leadptd, "sublead": subleadptd, "weight": weightsd})
roofd.close()
print('Successful close')

Running a total of  12205790  events
5M completed in 7 min 3 s
Approx time for finishing is 17 min 12 s
10M completed in 14 min 15 s
Approx time for finishing is 17 min 24 s
Initial events: 12205790
Final events: 12131468
Finished in 16 min 58 s
Successful close
